In [2]:
"""
Notebook basics.

This notebook shows you the basic steps for loading and displaying data
from checklists and observations. It assumes you have completed he steps in
the "Getting Started" section of the README, and you have an eBird API key.
"""
from django.conf import settings
from django.template.loader import render_to_string

from faker import Faker
from IPython.display import display, HTML

from ebird.api import get_visits
from checklists.loaders import APILoader
from checklists.models import Checklist, Observation

# Randomly choose a country to load checklists for.
region = Faker().country_code()

In [3]:
# Load the most recent checklists.
loader = APILoader(settings.EBIRD_API_KEY, locale=settings.EBIRD_API_LOCALE)

# Normally you use the APILoader for this, however if the country
# chosen at random is a big birding nation, then we're going to
# overload the eBird servers with requests. This way is safer.
# Get the visits from the API, the default number is 10.
visits = get_visits(settings.EBIRD_API_KEY, region)

for visit in visits:
    loader.add_location(visit["loc"])
    loader.load_checklist(visit["subId"])

2025-04-06 12:42 [INFO] Loading checklist: S223301417
2025-04-06 12:42 [INFO] Loading species: rocpig1
2025-04-06 12:42 [INFO] Fetching species: rocpig1, en_UK
2025-04-06 12:42 [INFO] Loading species: aspswi1
2025-04-06 12:42 [INFO] Fetching species: aspswi1, en_UK
2025-04-06 12:42 [INFO] Loading species: eutspa
2025-04-06 12:42 [INFO] Fetching species: eutspa, en_UK
2025-04-06 12:42 [INFO] Loading checklist: S223294275
2025-04-06 12:42 [INFO] Loading species: grecou1
2025-04-06 12:42 [INFO] Fetching species: grecou1, en_UK
2025-04-06 12:42 [INFO] Loading species: grbmal1
2025-04-06 12:42 [INFO] Fetching species: grbmal1, en_UK
2025-04-06 12:42 [INFO] Loading species: shikra1
2025-04-06 12:42 [INFO] Fetching species: shikra1, en_UK
2025-04-06 12:42 [INFO] Loading species: asbowl1
2025-04-06 12:42 [INFO] Fetching species: asbowl1, en_UK
2025-04-06 12:42 [INFO] Loading species: indrol3
2025-04-06 12:42 [INFO] Fetching species: indrol3, en_UK
2025-04-06 12:42 [INFO] Loading species: linba

In [5]:
# Show the 10 most recent checklists.
checklists = Checklist.objects.filter(location__country__code=region)

table = render_to_string("checklists.html", {"checklists": checklists})
display(HTML(table))

Identifier,Date,Time,Location,Region,Country
S223301417,"April 6, 2025",1:22 p.m.,Phonsavan,,Laos
S223294275,"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
S223294325,"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
S223036694,"April 5, 2025",12:20 p.m.,Xayaboury--Elephant Conservation Center,,Laos
S223320014,"April 5, 2025",12:20 p.m.,Xayaboury--Elephant Conservation Center,,Laos
S223015486,"April 5, 2025",12:02 p.m.,Xayaboury--Elephant Conservation Center,,Laos
S223294394,"April 5, 2025",11:18 a.m.,Xayaboury--Elephant Conservation Center,,Laos
S223018713,"April 5, 2025",10:41 a.m.,Tad Ka Waterfall,,Laos
S222809524,"April 4, 2025",5:02 p.m.,ວິທະຍາໄລຄູຄັງໄຂ,,Laos
S222777500,"April 4, 2025",9:41 a.m.,Plain of Jars--Site 1 (Phonsavan),,Laos


In [7]:
# Show the highest count for each species seen
# Rather than build a complex query to group the observations by species
# and calculate the highest count, we cheat by using the {% ifchanged %}
# tag in the template to display the observation each time the species
# changes. Sorting by count descending means this record is the one where
# the highest count was recorded.
observations = Observation.objects.filter(location__country__code=region).order_by("species__taxon_order", "-count")

table = render_to_string("highest-counts.html", {"observations": observations})
display(HTML(table))

Count,Species,Date,Time,Location,Region,Country
54,Lesser Whistling-Duck (Dendrocygna javanica),"April 5, 2025",11:18 a.m.,Xayaboury--Elephant Conservation Center,,Laos
1,Feral Pigeon (Columba livia (Feral Pigeon)),"April 6, 2025",1:22 p.m.,Phonsavan,,Laos
5,Greater Coucal (Centropus sinensis),"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
2,Green-billed Malkoha (Phaenicophaeus tristis),"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
1,Plaintive Cuckoo (Cacomantis merulinus),"April 5, 2025",12:20 p.m.,Xayaboury--Elephant Conservation Center,,Laos
16,Asian Palm Swift (Cypsiurus balasiensis),"April 6, 2025",1:22 p.m.,Phonsavan,,Laos
1,Chinese Pond Heron (Ardeola bacchus),"April 5, 2025",12:20 p.m.,Xayaboury--Elephant Conservation Center,,Laos
4,Black Baza (Aviceda leuphotes),"April 5, 2025",12:20 p.m.,Xayaboury--Elephant Conservation Center,,Laos
2,Shikra (Tachyspiza badia),"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
2,Asian Barred Owlet (Glaucidium cuculoides),"April 6, 2025",9:43 a.m.,Xayaboury--Elephant Conservation Center,,Laos
